## Домашнее задание №1
### Выполнили: Алексеев Антон, Булатов Артур, Корнеева Елизавета ИАД1

Итак, начнем по порядку.

#### Краткая постановка задачи и формулировка задания:

Домашнее задание заключается в получении и обработки данных (прогноз погоды на 10 дней), построении шаблонов, которые, исходя из обработанных данных, генерируют синтаксически верный, связный прогноз в текстовом формате. В заключении предлагается ответить на ряд вопросов по результатам проделанной

#### Описание используемых инструментов:

pandas — программная библиотека на языке Python для обработки и анализа данных. Pandas предоставляет специальные структуры данных и операции для манипулирования числовыми таблицами и временными рядами.

lxml это быстрая и гибкая библиотека для обработки разметки XML и HTML на Python. Кроме того, в ней присутствует возможность разложения элементов документа в дерево.

pymorphy2 - морфологический анализатор для русского языка, написанный на языке Python и использующий словари из OpenCorpora. Он работает под Python 2.6, 2.7, 3.2 и 3.3 и распространяется по лицензии MIT.

#### Краткий план работы программы:


   - Часть первая "Сбор данных"
        - Скачиваем код со страницы gismeteo
        - Строим lxml.html дерево
        - Находим нужные нам элементы
        - Заполняем таблицу
   - Часть вторая "Генератор описания прогноза погоды"
        - Первый Template
            - Выбираем 2 дня
            - Усредняем минимальную и максимальную температуры за оба дня
            - Находим разницу
            - Согласуем глаголы, существительные, предлоги и т.д.
            - Получаем итоговую строку
            - Выводим описания погоды по первому шаблону
        - Второй Template
            - Выбираем 2 дня
            - Находим разницу в показании ветра
            - Согласуем глаголы, существительные, предлоги и т.д.
            - Получаем итоговую строку
            - Выводим описания погоды по второму шаблону
        - Третий Template
            - Выбираем 2 дня
            - Находим разницу в показании осадков
            - Согласуем глаголы, существительные, предлоги и т.д.
            - Получаем итоговую строку
            - Выводим описания погоды по третьему шаблону
   - Часть третья "Ответьте на вопросы"
        - Отвечаем на вопросы

Более подробно каждый шаг расписан в виде комментариев к коду

### Если библиотеки уже установлены, то пропустите следующую клетку

In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install pymorphy2

In [2]:
# Импортируем необходимые библиотеки
from urllib import request
import lxml.html
import pandas as pd
import datetime
import pymorphy2

# Часть первая "Сбор данных"

In [3]:
# Скачиваем html-код страницы прогноза погоды во Владивостоке и создаем объект класса lmxl.html для
# получения и обработки необходимой информации
url = "https://www.gismeteo.ru/weather-vladivostok-4877/10-days/"
output = request.urlopen(url).read().decode("utf-8") 
html = lxml.html.fromstring(output)

In [4]:
# Заменяет тире на минус в числах и запятую на точку в датах для удобного парсинга
def r(text):
    return text.replace('−', '-').replace(',','.')

In [5]:
# Выбираем ноды для каждого типа информации (температура, скорость ветра, осадки)
temperatures = html.find_class('value')
precipitations = html.find_class('w_precipitation__value')
winds = html.find_class('w_wind')
dates = html.find_class('w_date')

In [6]:
# Инициализируем вспомогательные объекты
data = {}
indexes = ["максимальная температура","минимальная температура","скорость ветра","уровень осадков"]
columns = []
m = datetime.date.today()
morph = pymorphy2.MorphAnalyzer()
days = {'Пн' : 'понедельник', 
       'Вт' : 'вторник',
        'Ср' : 'среда', 
        'Чт' : 'четверг', 
        'Пт' : 'пятница', 
        'Сб' : 'суббота', 
        'Вс' : 'воскресенье'
       }

In [7]:
# Заполняем данные
for i in range(10):
    col = dates[i][0][1].text.strip().split(' ')[0]+"."+m.strftime('%m')+" "+dates[i][0][0].text
    columns.append(col)
    temp_min = int(r(temperatures[i][0].text))
    temp_max = int(r(temperatures[i][1].text))
    wind = int(str(lxml.html.tostring(winds[i][0][0])).split('>')[-1].split('\\n')[1].strip())
    # Вот тут случайно нашли интересный баг. Если за все 10 дней осадков в городе не предвидится совем, 
    # то gismeteo.ru скажет нам "нет осадков" и не оставит нам 10 нод с нулями, а оставит пустой массив precipitations, поэтому при анализе осадков в некоторых городах прога могла бы падать"
    precipitation = float(r(precipitations[i].text.strip())) if len(precipitations)>0 else 0
    data[col] = [temp_min,temp_max,wind,precipitation]

In [8]:
# Создаем DataFrame на основе полученных данных
df = pd.DataFrame(data,index=indexes)

In [9]:
# Визуализируем таблицу
df

,18.02 Вс,19.02 Пн,20.02 Вт,21.02 Ср,22.02 Чт,23.02 Пт,24.02 Сб,25.02 Вс,26.02 Пн,27.02 Вт
максимальная температура,-4.0,-2.0,-1.0,-2.0,0.0,-3.0,-5.0,-5.0,-3.0,-3.0
минимальная температура,-11.0,-9.0,-9.0,-10.0,-6.0,-8.0,-10.0,-10.0,-10.0,-7.0
скорость ветра,14.0,10.0,15.0,15.0,12.0,13.0,12.0,8.0,9.0,15.0
уровень осадков,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4


# Часть вторая "Генератор описания прогноза погоды"

In [10]:
# Template, генерирующий текст вида "В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2"
def temperature_template(day1_date, day2_date):
    
    day1_name = days[day1_date[-2:]]
    day1_name = morph.parse(day1_name)[0]
    day1_name = day1_name.inflect({'accs'}).word
    
    day2_name = days[day2_date[-2:]]
    day2_name = morph.parse(day2_name)[0]
    day2_name = day2_name.inflect({'ablt'}).word
    
    temp1 = (df[day1_date][0] + df[day1_date][1]) // 2
    temp2 = (df[day2_date][0] + df[day2_date][1]) // 2
    
    diff = abs(temp1 - temp2)
    
    degree = 'градус'
    degree = morph.parse(degree)[0]
    degree = degree.make_agree_with_number(diff).word
    
    if (temp1 > temp2):
        verb = 'потеплеет'
    else:
        verb = 'похолодает'
        
    city = 'Владивосток'
    city = morph.parse(city)[0]
    city = city.inflect({'loc2'}).word
    
    
    if(diff == 0):
        s = ' '.join(['В', day1_name, day1_date.split(' ')[0], 'во', city.title(), "будет такая же температура", 'по сравнению с', day2_name, day2_date.split(' ')[0]])
    else:
        s = ' '.join(['В', day1_name, day1_date.split(' ')[0], 'во', city.title(), verb, 'на', str(int(diff)), degree, 'по сравнению с', day2_name, day2_date.split(' ')[0]])
        
    return s
    

In [11]:
# Если вы хотите проверить модель на определённых днях - раскомментируйте следующую строку
#print(temperature_template('18.02 Вс', '17.02 Сб'))
for i in range(9,0,-1):
    for j in range(i-1,-1,-1):
        print(temperature_template(columns[i], columns[j]))

В вторник 27.02 во Владивостоке потеплеет на 2 градуса по сравнению с понедельником 26.02
В вторник 27.02 во Владивостоке потеплеет на 3 градуса по сравнению с воскресеньем 25.02
В вторник 27.02 во Владивостоке потеплеет на 3 градуса по сравнению с субботой 24.02
В вторник 27.02 во Владивостоке потеплеет на 1 градус по сравнению с пятницей 23.02
В вторник 27.02 во Владивостоке похолодает на 2 градуса по сравнению с четвергом 22.02
В вторник 27.02 во Владивостоке потеплеет на 1 градус по сравнению с средой 21.02
В вторник 27.02 во Владивостоке будет такая же температура по сравнению с вторником 20.02
В вторник 27.02 во Владивостоке потеплеет на 1 градус по сравнению с понедельником 19.02
В вторник 27.02 во Владивостоке потеплеет на 3 градуса по сравнению с воскресеньем 18.02
В понедельник 26.02 во Владивостоке потеплеет на 1 градус по сравнению с воскресеньем 25.02
В понедельник 26.02 во Владивостоке потеплеет на 1 градус по сравнению с субботой 24.02
В понедельник 26.02 во Владивостоке

In [12]:
# Template, генерирующий текст вида "Скорость ветра изменится на X единиц в день1 по сравнению с день2"
def wind_template(day1_date, day2_date):
    
    day1_name = days[day1_date[-2:]]
    day1_name = morph.parse(day1_name)[0]
    day1_name = day1_name.inflect({'accs'}).word
    
    day2_name = days[day2_date[-2:]]
    day2_name = morph.parse(day2_name)[0]
    day2_name = day2_name.inflect({'ablt'}).word
    
    wind1 = df[day1_date][2]
    wind2 = df[day2_date][2]
    
    diff = abs(wind1 - wind2)
    
    unit = 'единица'
    unit = morph.parse(unit)[0]
    unit = unit.inflect({'accs'}).make_agree_with_number(diff).word
    
    if(diff == 0):
        s = ' '.join(['Скорость ветра не изменится', 'в', day1_name, day1_date.split(' ')[0], 'по сравнению с', day2_name,day2_date.split(' ')[0]])
    else:
        s = ' '.join(['Скорость ветра изменится на', str(int(diff)), unit, 'в', day1_name, day1_date.split(' ')[0], 'по сравнению с', day2_name,day2_date.split(' ')[0]])
        
    return s
    

In [13]:
#Если вы хотите проверить модель на определённых днях - раскомментируйте следующую строку
#print(wind_template('18.02 Вс', '17.02 Сб'))
for i in range(9,0,-1):
    for j in range(i-1,-1,-1):
        print(wind_template(columns[i], columns[j]))

Скорость ветра изменится на 6 единиц в вторник 27.02 по сравнению с понедельником 26.02
Скорость ветра изменится на 7 единиц в вторник 27.02 по сравнению с воскресеньем 25.02
Скорость ветра изменится на 3 единицы в вторник 27.02 по сравнению с субботой 24.02
Скорость ветра изменится на 2 единицы в вторник 27.02 по сравнению с пятницей 23.02
Скорость ветра изменится на 3 единицы в вторник 27.02 по сравнению с четвергом 22.02
Скорость ветра не изменится в вторник 27.02 по сравнению с средой 21.02
Скорость ветра не изменится в вторник 27.02 по сравнению с вторником 20.02
Скорость ветра изменится на 5 единиц в вторник 27.02 по сравнению с понедельником 19.02
Скорость ветра изменится на 1 единицу в вторник 27.02 по сравнению с воскресеньем 18.02
Скорость ветра изменится на 1 единицу в понедельник 26.02 по сравнению с воскресеньем 25.02
Скорость ветра изменится на 3 единицы в понедельник 26.02 по сравнению с субботой 24.02
Скорость ветра изменится на 4 единицы в понедельник 26.02 по сравнени

In [14]:
# Template, генерирующий текст вида "Уровень осадков повысится / понизится на X единиц за Y дней"
from datetime import datetime
def precipitation_template(day1_date, day2_date):

    date_format = "%d.%m"
    day1 = datetime.strptime(day1_date[:5], date_format)
    day2 = datetime.strptime(day2_date[:5], date_format)
    
    delta = int((day1 - day2).days)

    prec1 = df[day1_date][3]
    prec2 = df[day2_date][3]
    
    diff = abs(prec1 - prec2)
    
    unit = 'единица'
    unit = morph.parse(unit)[0]
    # Если число меньше 1, то согласуется оно так же, как и число, которое в 10 раз больше него
    unit = unit.inflect({'accs'}).make_agree_with_number(diff if diff>1 else diff*10).word
    
    days_count = 'дни'
    days_count = morph.parse(days_count)[0]
    days_count = days_count.make_agree_with_number(delta).word
    
    if (prec1 > prec2):
        verb = 'повысится'
    else:
        verb = 'понизится'
    
    if(diff == 0):
        s = ' '.join(['Уровень осадков', 'не изменится', 'за', str(delta), days_count, '(с', day2_date.split(' ')[0],'по', day1_date.split(' ')[0]+')'])
    else:
        s = ' '.join(['Уровень осадков', verb, 'на', str(diff), unit, 'за', str(delta), days_count, '(с', day2_date.split(' ')[0],'по', day1_date.split(' ')[0]+')'])
        
    return s

In [15]:
#Если вы хотите проверить модель на определённых днях - раскомментируйте следующую строку
#print(precipitation_template('18.02 Вс', '17.02 Сб'))
for i in range(9,0,-1):
    for j in range(i-1,-1,-1):
        print(precipitation_template(columns[i], columns[j]))

Уровень осадков повысится на 0.4 единицы за 1 день (с 26.02 по 27.02)
Уровень осадков повысится на 0.4 единицы за 2 дня (с 25.02 по 27.02)
Уровень осадков повысится на 0.4 единицы за 3 дня (с 24.02 по 27.02)
Уровень осадков повысится на 0.4 единицы за 4 дня (с 23.02 по 27.02)
Уровень осадков повысится на 0.4 единицы за 5 дней (с 22.02 по 27.02)
Уровень осадков повысится на 0.4 единицы за 6 дней (с 21.02 по 27.02)
Уровень осадков повысится на 0.4 единицы за 7 дней (с 20.02 по 27.02)
Уровень осадков повысится на 0.4 единицы за 8 дней (с 19.02 по 27.02)
Уровень осадков повысится на 0.4 единицы за 9 дней (с 18.02 по 27.02)
Уровень осадков не изменится за 1 день (с 25.02 по 26.02)
Уровень осадков не изменится за 2 дня (с 24.02 по 26.02)
Уровень осадков не изменится за 3 дня (с 23.02 по 26.02)
Уровень осадков не изменится за 4 дня (с 22.02 по 26.02)
Уровень осадков не изменится за 5 дней (с 21.02 по 26.02)
Уровень осадков не изменится за 6 дней (с 20.02 по 26.02)
Уровень осадков не изменится

# Часть третья "Ответьте на вопросы"

#### Вопрос первый: 
- В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?

#### Ответ:

Генераторы текстов широко используются при разработке и поисковой оптимизации сайтов: для генерации названий, описаний, и содержимого целых сайтов с помощью доргенов (генераторов дорвеев).

Существуют крупные англоязычные сайты, зарабатывающие на размещении рекламы, на которых весь контент пишут не журналисты, а боты — статьи автоматическим рерайтятся из других источников. Русский язык, в отличие от английского, имеет сложную морфологию, поэтому появление подобных ботов-рерайтеров в рунете сильно осложнено.

Нельзя не вспомнить про виртуальных собеседников (чат-боты), получивших в последнее время немалую популярность. Это программы, предназначенные имитировать общение в чатах. Они массово применяются для рассылки спама в соцсетях (спам-боты), а также как автоответчики, способные реагировать на множества ключевых слов по разным сценариям.

Что касается генерации текста об изменении числовых показателей по шаблонам, то тут сразу в голову приходит идея про спортивного (футбольного, баскетбольного, хоккейного и т.д.) бота, способного генерировать текстовую онлайн трансляцию на основе изменения счёта по ходу матча.

#### Вопрос второй:
- Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче?

#### Ответ:

Одним из способов разнообразить шаблоны является анализ входных параметров для шаблона. Приведу пример:
Если просто подставить в шаблон "Уровень осадков повысится / понизится на X единиц за Y дней" вместа X 0, то полученное предложение будет понятно, но недостаточно красивым. Совсем другое дело, когда можно понять, что 0, это значит, что изменений нет. В таком случае совесем небольшое изменение делает Template более универсальным.
Прошу заметить, что в ходе ответа на этот вопрос мы осознали и учли эту небольшую хитрость. Соответственно ввнесли правки в код Template. Вот что получилось в случае X = 0 : Уровень осадков не изменится за 7 дней (с 18.02 по 25.02). Аналогичная ситуация с температурой, где похолодает/потеплеет на 0 градусов звучит не очень.

Знание синтаксической структуры предложения помогает нам с помощью знания языка согласовывать падежи, предлоги т.д.

# Результаты работы

- В ходе этого увлекательного практического исследования мы использовали уже изученные(urllib и pandas) и совершенно новые(pymorphy2 и lxml) библиотеки для получения данных, обработки данных и построения шаблонов. 
- Научились генерировать синтаксически верные, связные тексты по шаблону. 
- С творческим подходом подошли к ответам на заданные нам вопросы. 
- Нашли и исправили различные интересные баги (которые появились не по нашей вине, они описаны в комментариях к коду)(которые появились по нашей вине - тоже нашли и исправили). 
- Улучшили шаблоны для более универсальной работы.